<a href="https://colab.research.google.com/github/pashok3d/RemarqueGPT/blob/main/RemarqueGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Building GPT from scratch and training it on all books of Erich Maria Remarque
"""

'\nBuilding GPT from scratch and training it on all books of Erich Maria Remarque\n'

In [2]:
!pip install tqdm -q
!pip install wandb -q
!pip install tiktoken -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.0 MB/s eta 0:00:00


In [3]:
!mkdir dataset
!mkdir model

In [4]:
import wandb
import torch
import math
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import ConcatDataset, Dataset
from torch import nn
from transformers import get_linear_schedule_with_warmup
from torch.nn.utils import clip_grad_norm_

In [5]:
from tokenizers import (
    Tokenizer,
    decoders,
    models,
    pre_tokenizers,
    trainers,
    processors,
    normalizers,
)

In [6]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
WINDOW_SIZE = 256
BATCH_SIZE = 512
EPOCHS = 10
LR = 1e-4
EMBEDDING_DIM = 128
BLOCKS_NUM = 4
HEADS_NUM = 4
DROPOUT = 0.2
VOCAB_SIZE = 2048
MAX_GRAD_NORM = 1.0
VALIDATION_INTERVAL = 1000

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

config = {
    "learning_rate": LR,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "window_size": WINDOW_SIZE,
    "embedding_dim": EMBEDDING_DIM,
    "blocks_num": BLOCKS_NUM,
    "heads_num": HEADS_NUM,
    "dropout": DROPOUT,
    "vocab_size": VOCAB_SIZE,
    "max_grad_norm": MAX_GRAD_NORM,
    "device": device,
}

In [10]:
run = wandb.init(project="remark-gpt", config=config)

wandb: Currently logged in as: crush_tarash. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [11]:
def generate_text(
    model,
    tokenizer: Tokenizer,
    prompt: str,
    device: str,
    window_size: int,
    max_tokens: int = 10,
    temperature: float = 1.0,
) -> str:
    """Generate text using the trained GPT model."""
    model.eval()
    context = tokenizer.encode(prompt).ids
    generated = list(context)

    with torch.no_grad():
        for _ in range(max_tokens):
            x = torch.tensor(context[-window_size:]).unsqueeze(0).to(device)
            logits, _ = model(x)
            logits = logits[0, -1, :] / temperature
            probs = torch.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1).item()
            generated.append(next_token)
            context = generated

    return tokenizer.decode(generated)


class TextDataset(Dataset):
    def __init__(self, path, context_window_size, tokenizer: Tokenizer):
        # Load dataset
        with open(path, "r") as f:
            lines = f.readlines()

        text = "\n".join(lines)

        self.tokens = tokenizer.encode(text).ids

        self.x = []
        self.y = []
        for i in range(len(self.tokens) - context_window_size):
            self.x.append(self.tokens[i : i + context_window_size])
            self.y.append(self.tokens[i + 1 : i + context_window_size + 1])

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]), torch.tensor(self.y[idx])


def get_datasets(paths, context_window_size, tokenizer: Tokenizer):
    datasets = []

    for path in paths:
        dataset = TextDataset(path, context_window_size, tokenizer)
        datasets.append(dataset)

    return datasets


In [12]:
# Prepare tokenizer
dataset_lines = []

dataset_paths = [
    "dataset/All_Quiet_on_the_Western_Front_1929_AST_978-5-17-105639-1.txt",
    "dataset/All_Quiet_on_the_Western_Front_1929_AST_978-5-17-137374-0.txt",
    "dataset/Station_at_the_Horizon_1928_AST_978-5-17-133322-5.txt",
    "dataset/The_Dream_Room_1920_AST_978-5-17-071518-3.txt",
    "dataset/The_Road_Back_1931.txt",
    "dataset/Tree_Comrades_1936_978-5-17-004252-4.txt",
    "dataset/Tree_Comrades_1936_978-5-17-056963-2.txt",
    "dataset/Tree_Comrades_1936_978-5-17-108545-2.txt"
]

for path in [ds_path.replace(".txt", "-train.txt") for ds_path in dataset_paths]:
    with open(path, "r") as f:
        dataset_lines.extend(f.readlines())

alphabet = list(set("".join(dataset_lines)))

tokenizer = Tokenizer(models.BPE())
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE)
tokenizer.normalizer = normalizers.Lowercase()
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()
tokenizer.train_from_iterator(dataset_lines, trainer=trainer)

ds_list = get_datasets(
    [ds_path.replace(".txt", "-train.txt") for ds_path in dataset_paths],
    WINDOW_SIZE,
    tokenizer,
)

train_ds = ConcatDataset(ds_list)
train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

dev_dataset_paths = [ds_path.replace(".txt", "-dev.txt") for ds_path in dataset_paths]

dev_ds_list = get_datasets(dev_dataset_paths,
    WINDOW_SIZE,
    tokenizer,
)

names_with_dev_dataloaders = []

for i, dev_ds in enumerate(dev_ds_list):
    dev_dataloader = DataLoader(dev_ds, batch_size=BATCH_SIZE, shuffle=False)
    names_with_dev_dataloaders.append(
        (dev_dataset_paths[i].replace("dataset/", "").replace(".txt", ""), dev_dataloader)
    )

In [14]:
class AttentionHead(nn.Module):
    def __init__(self, embedding_dim, head_dim, dropout, max_len):
        super().__init__()

        self.Q = nn.Linear(embedding_dim, head_dim)
        self.K = nn.Linear(embedding_dim, head_dim)
        self.V = nn.Linear(embedding_dim, head_dim)

        self.kv_softmax = nn.Softmax(dim=-1)
        self.attn_dropout = nn.Dropout(dropout)

        self.register_buffer("tril", torch.tril(torch.ones(max_len, max_len)))

    def forward(self, norm_inputs):

        _, T, _ = norm_inputs.shape

        q = self.Q(norm_inputs)
        k = self.K(norm_inputs)
        v = self.V(norm_inputs)

        attention_weights = (q @ k.transpose(-1, -2)) / math.sqrt(
            q.shape[-1]
        )  # shape: (B, T, T)
        attention_weights_masked = attention_weights.masked_fill(
            self.tril[:T, :T] == 0, -torch.inf
        )
        attention_scores = self.kv_softmax(attention_weights_masked)
        attention_scores = self.attn_dropout(attention_scores)

        return attention_scores @ v


class GPTBlock(nn.Module):
    def __init__(
        self, embedding_dim: int, max_len: int, dropout: float = 0.1, n_heads=2
    ):
        super().__init__()
        head_dim = embedding_dim // n_heads

        # Attention heads
        self.heads = nn.ModuleList(
            [
                AttentionHead(embedding_dim, head_dim, dropout, max_len)
                for _ in range(n_heads)
            ]
        )

        # Feedforward
        self.f1 = nn.Linear(embedding_dim, embedding_dim * 4)
        self.f_act = nn.SiLU()
        self.f2 = nn.Linear(embedding_dim * 4, embedding_dim)
        self.ff_dropout = nn.Dropout(dropout)

        self.ln1 = nn.LayerNorm(embedding_dim)
        self.ln2 = nn.LayerNorm(embedding_dim)

    def forward(self, inputs):
        norm_inputs = self.ln1(inputs)
        attn_out = torch.concat([h(norm_inputs) for h in self.heads], dim=-1)

        # Add residual connection
        x = inputs + attn_out

        norm_x = self.ln2(x)
        ff_out = self.ff_dropout(self.f2(self.f_act(self.f1(norm_x))))
        out = x + ff_out
        return out


class GPT(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        max_len: int,
        embedding_dim: int = 16,
        blocks_num: int = 4,
        n_heads: int = 2,
        dropout: float = 0.1,
    ):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.emb = nn.Embedding(vocab_size, embedding_dim)
        self.pos = nn.Embedding(max_len, embedding_dim)
        self.blocks = nn.Sequential(
            *[
                GPTBlock(embedding_dim, max_len, dropout, n_heads)
                for _ in range(blocks_num)
            ]
        )
        self.proj = nn.Linear(embedding_dim, vocab_size)
        self.loss_fn = nn.CrossEntropyLoss()
        self.emb_dropout = nn.Dropout(dropout)

    def forward(self, inputs, labels=None):
        _, T = inputs.shape
        embs = self.emb(inputs)
        pos_embs = self.pos(torch.arange(T, device=inputs.device))  # (T,C)
        blocks_output = self.emb_dropout(self.blocks(embs + pos_embs))
        logits = self.proj(blocks_output)  # (B,T,vocab_size)
        if labels is not None:
            loss = self.loss_fn(logits.view(-1, self.vocab_size), labels.view(-1))
            return logits, loss
        else:
            return logits, None


In [15]:
model = GPT(vocab_size=tokenizer.get_vocab_size(), max_len=WINDOW_SIZE, embedding_dim=EMBEDDING_DIM, blocks_num=BLOCKS_NUM, n_heads=HEADS_NUM, dropout=DROPOUT)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

# Scheduler with warmup and linear decay
TOTAL_STEPS = EPOCHS * len(train_dataloader)
WARMUP_STEPS = int(0.05 * TOTAL_STEPS)  # 5% warmup

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=TOTAL_STEPS)

In [ ]:
wandb.watch(model, log_freq=2500, log='all')

In [ ]:
# model.train()
# epoch_loss = 0
# steps_n = 0
# with torch.no_grad():
#     for batch in tqdm(train_dataloader):
#         input, labels = batch[0].to(device), batch[1].to(device)
#         output, loss = model(input, labels)
#         epoch_loss += loss.item()
#         steps_n += 1
#     avg_loss = epoch_loss / steps_n
# expected_init_loss = -math.log(1 / tokenizer.get_vocab_size())
# print(f"initial train loss: {avg_loss:.3f}, with expected of {expected_init_loss:.3f}")

In [16]:
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    steps_n = 0
    total_steps = 0

    for batch in tqdm(train_dataloader):
        input, labels = batch[0].to(device), batch[1].to(device)
        output, loss = model(input, labels)
        loss.backward()
        clip_grad_norm_(model.parameters(), max_norm=MAX_GRAD_NORM)
        optimizer.step()
        scheduler.step()  # Update the learning rate
        optimizer.zero_grad()
        epoch_loss += loss.item()
        total_steps += 1
        steps_n += 1
        run.log({"train_loss": loss.item(), "lr": scheduler.get_last_lr()[0]})

        if total_steps % VALIDATION_INTERVAL == 0:
            model.eval()
            with torch.no_grad():
                for ds_name, dev_dataloader in names_with_dev_dataloaders:
                    val_steps_n = 0
                    val_epoch_loss = 0
                    for batch in dev_dataloader:
                        input, labels = batch[0].to(device), batch[1].to(device)
                        output, loss = model(input, labels)
                        val_epoch_loss += loss.item()
                        val_steps_n += 1
                    avg_val_loss = val_epoch_loss / val_steps_n
                    print(f"val loss for {ds_name}: {avg_val_loss:.3f}")
                    metric_name = f"avg_val_loss_{ds_name}"
                    run.log({metric_name: avg_val_loss}, commit=False)
            model.train()

  0%|          | 0/2170 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 107.06 MiB is free. Process 3049 has 14.64 GiB memory in use. Of the allocated memory 14.41 GiB is allocated by PyTorch, and 108.74 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
torch.save(model.state_dict(), "model/gpt.pt")

In [ ]:
artifact = wandb.Artifact("model", type="model")
artifact.add_file("model/gpt.pt")
run.log_artifact(artifact)

In [ ]:
wandb.finish()

In [ ]:
model.eval()
prompt = "Привет, любовь моя"
generated_text = generate_text(model, tokenizer, prompt, device, WINDOW_SIZE, max_tokens=250, temperature=0.7)

In [ ]:
print(generated_text)